In [11]:
from tensorflow import keras
batch_size = 32
num_classes = 10
epochs = 10

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D

model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=[32,32,3]))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(256, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))
#model.summary()

W0811 13:17:38.598859 16812 deprecation.py:506] From C:\Users\Skyno\Anaconda3\envs\tensorflow-gpu\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [3]:
model.save_weights("tmp.h5")


In [4]:
from tensorflow.keras.datasets import cifar10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [5]:
def cifar_generator(total_size, batch_size=32):
    while True:
        for indexs in range(0, total_size, batch_size):
            images = x_train[indexs: indexs+batch_size]
            labels = y_train[indexs: indexs+batch_size]
            yield images, labels
            
cifar_gen = cifar_generator(len(x_train), batch_size)

In [12]:
model.load_weights("tmp.h5")
model.compile(loss='categorical_crossentropy',
              optimizer=keras.optimizers.SGD(0.01, 0.9),
              metrics=['accuracy'])

history = model.fit_generator(cifar_gen,
                              steps_per_epoch=len(x_train)//batch_size,
                              epochs=epochs,
                              verbose=1,
                              validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/10
1562/1562 [==============================] - 22s 14ms/step - loss: 1.9363 - acc: 0.2767 - val_loss: 1.6862 - val_acc: 0.3745
Epoch 2/10
1562/1562 [==============================] - 21s 13ms/step - loss: 1.6299 - acc: 0.4003 - val_loss: 1.3819 - val_acc: 0.4997
Epoch 3/10
1562/1562 [==============================] - 21s 13ms/step - loss: 1.4699 - acc: 0.4632 - val_loss: 1.3112 - val_acc: 0.5348
Epoch 4/10
1562/1562 [==============================] - 21s 13ms/step - loss: 1.3452 - acc: 0.5148 - val_loss: 1.1588 - val_acc: 0.5849
Epoch 5/10
1562/1562 [==============================] - 21s 13ms/step - loss: 1.2615 - acc: 0.5481 - val_loss: 1.0371 - val_acc: 0.6308
Epoch 6/10
1562/1562 [==============================] - 21s 14ms/step - loss: 1.1879 - acc: 0.5763 - val_loss: 1.0183 - val_acc: 0.6340
Epoch 7/10
1562/1562 [==============================] - 21s 13ms/step - loss: 1.1405 - acc: 0.5980 - val_loss: 0.9222 - val_acc: 0.6743
Epoch 8/10
1562/1562 [==========================

In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

data_generator = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

In [8]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
x_test = x_test.astype('float32')
x_test /= 255

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [9]:
def cifar_aug_generator(total_size, batch_size=32):
    while True:
        for indexs in range(0, total_size, batch_size):
            images = x_train[indexs: indexs+batch_size]
            labels = y_train[indexs: indexs+batch_size]
            images = next(data_generator.flow(images, shuffle=False))
            images = images.astype('float32')
            images /= 255
            yield images, labels
             
cifar_gen = cifar_aug_generator(len(x_train), batch_size)

In [13]:

model.load_weights("tmp.h5")
model.compile(loss='categorical_crossentropy',
              optimizer=keras.optimizers.SGD(0.01, 0.9),
              metrics=['accuracy'])

history = model.fit_generator(cifar_gen,
                              steps_per_epoch=len(x_train)//batch_size,
                              epochs=epochs,
                              verbose=1,
                              validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/10
1562/1562 [==============================] - 21s 13ms/step - loss: 1.9411 - acc: 0.2740 - val_loss: 1.5812 - val_acc: 0.4063
Epoch 2/10
1562/1562 [==============================] - 21s 13ms/step - loss: 1.6291 - acc: 0.3997 - val_loss: 1.2884 - val_acc: 0.5319
Epoch 3/10
1562/1562 [==============================] - 20s 13ms/step - loss: 1.4611 - acc: 0.4678 - val_loss: 1.2374 - val_acc: 0.5642
Epoch 4/10
1562/1562 [==============================] - 20s 13ms/step - loss: 1.3488 - acc: 0.5137 - val_loss: 1.1585 - val_acc: 0.5950
Epoch 5/10
1562/1562 [==============================] - 20s 13ms/step - loss: 1.2678 - acc: 0.5439 - val_loss: 1.1219 - val_acc: 0.6128
Epoch 6/10
1562/1562 [==============================] - 20s 13ms/step - loss: 1.1941 - acc: 0.5746 - val_loss: 1.0215 - val_acc: 0.6360
Epoch 7/10
1562/1562 [==============================] - 21s 13ms/step - loss: 1.1419 - acc: 0.5938 - val_loss: 1.0003 - val_acc: 0.6547
Epoch 8/10
1562/1562 [==========================